In [1]:
import warnings
#import math
#from IPython.display import display, HTML, SVG
import pandas as pd
import neutcurve
import altair as alt
import re
import os
#print(f"Using `neutcurve` version {neutcurve.__version__}")
import sys

# allow more rows for Altair
_ = alt.data_transformers.disable_max_rows()

#import altair theme and enable
sys.path.append('../../config/')
import theme
alt.themes.register('main_theme', theme.main_theme)
alt.themes.enable('main_theme')

ThemeRegistry.enable('main_theme')

In [2]:
neut_file_path = '../../data/files/niv_rbp_antibody_neuts.csv'
fitParams_output = '../../results/fitparams.csv'
output_png_path = '../../results/ephrin_receptor.png'

In [3]:
receptor_flag = False
sera_flag = False
antibody_flag = True
vary_serum_flag = True
vary_virus_flag = False

In [4]:
df = pd.read_csv(neut_file_path)

if 'serum' not in df.columns:
    print('serum is not a column')
elif 'virus' not in df.columns:
    print('virus is not a column')
elif 'replicate' not in df.columns:
    print('replicate is not a column')
elif 'concentration' not in df.columns:
    print('concentration is not a column')
elif 'fraction infectivity' not in df.columns:
    print('fraction infectivity is not a column')


#print(df.columns)
display(df.head(10))

,serum,virus,replicate,concentration,fraction infectivity
0,nAH1.3,NiV,1,2.50000,0.0000
1,nAH1.3,NiV,1,0.62500,0.0000
2,nAH1.3,NiV,1,0.15625,0.0000
3,nAH1.3,NiV,1,0.03906,0.2105
4,nAH1.3,NiV,1,0.00977,0.7908
5,nAH1.3,NiV,1,0.00244,0.9350
6,nAH1.3,NiV,1,0.00061,0.8966
7,nAH1.3,NiV,2,2.50000,0.0000
8,nAH1.3,NiV,2,0.62500,0.0000
9,nAH1.3,NiV,2,0.15625,0.0005


In [5]:
# Estimate neutralization curves using the `curvefits` module from `neutcurve` package.
def get_neutcurve(df, replicate="average"):
    #estimate fits
    fits = neutcurve.curvefits.CurveFits(
        data=df,
        serum_col="serum",
        virus_col="virus",
        replicate_col="replicate",
        conc_col="concentration",
        fracinf_col="fraction infectivity",
        fixbottom=0,
    )
    
    fitParams = fits.fitParams(ics=[50, 90, 99])

    #get list of different sera and viruses that were tested
    serum_list = list(df["serum"].unique())
    virus_list = list(df["virus"].unique())

    curves = [] #initialize an empty list to store neutralization curve data
    
    # Loop over each serum type and retrieve the curve
    for serum in serum_list:
        for virus in virus_list:
            curve = fits.getCurve(serum=serum, virus=virus, replicate=replicate)
            neut_df = curve.dataframe() #turn into a dataframe
            neut_df["serum"] = serum #assign serum name to a column
            neut_df["virus"] = virus #assign virus name to a column
            curves.append(neut_df)

    # Concatenate all the dataframes into one
    combined_curve = pd.concat(curves, axis=0)
    combined_curve["upper"] = combined_curve["measurement"] + combined_curve["stderr"]
    combined_curve["lower"] = combined_curve["measurement"] - combined_curve["stderr"]
    
    return combined_curve, fitParams


neutcurve_df,fitParams = get_neutcurve(df)
#display(neutcurve_df.head(3))

fitParams = fitParams.drop(['replicate','nreplicates'],axis=1)
import re

# Function to rename columns
def rename_ic_columns(col_name):
    match = re.match(r'(ic\d{2}$)', col_name)
    if match:
        return f"{match.group(1)}_ug"
    return col_name

# Rename columns
fitParams.rename(columns=rename_ic_columns, inplace=True)

# Function to create new 'ng' columns
def create_ng_columns(df):
    for col in df.columns:
        match = re.match(r'(ic\d{2})_ug', col)
        if match:
            new_col_name = f"{match.group(1)}_ng"
            print(new_col_name)
            df[new_col_name] = df[col].mul(1000).round(1)
    return df

fitParams = create_ng_columns(fitParams)
# Rename columns
#fitParams.rename(columns=rename_ic_columns, inplace=True)

display(fitParams)
#fitParams.to_csv(fitParams_output,index=False)

ic50_ng
ic90_ng
ic99_ng


,serum,virus,ic50_ug,ic50_bound,ic50_str,ic90_ug,ic90_bound,ic90_str,ic99_ug,ic99_bound,ic99_str,midpoint,slope,top,bottom,ic50_ng,ic90_ng,ic99_ng
0,nAH1.3,NiV,0.023109,interpolated,0.0231,0.073686,interpolated,0.0737,0.261203,interpolated,0.261,0.023109,1.894850,1,0,23.1,73.7,261.2
1,m102.4,NiV,0.012268,interpolated,0.0123,0.037291,interpolated,0.0373,0.125469,interpolated,0.125,0.012268,1.976310,1,0,12.3,37.3,125.5
2,HENV-26,NiV,0.013720,interpolated,0.0137,0.035035,interpolated,0.035,0.097458,interpolated,0.0975,0.013720,2.343804,1,0,13.7,35.0,97.5
3,HENV-32,NiV,0.142970,interpolated,0.143,0.556687,interpolated,0.557,2.454114,interpolated,2.45,0.142970,1.616357,1,0,143.0,556.7,2454.1
4,HENV-103,NiV,0.066669,interpolated,0.0667,0.191746,interpolated,0.192,0.607333,interpolated,0.607,0.066669,2.079869,1,0,66.7,191.7,607.3
5,HENV-117,NiV,0.011686,interpolated,0.0117,0.021845,interpolated,0.0218,0.043236,interpolated,0.0432,0.011686,3.512342,1,0,11.7,21.8,43.2
6,EFNB2-monomeric,NiV,0.053474,interpolated,0.0535,0.140630,interpolated,0.141,0.403987,interpolated,0.404,0.053474,2.272349,1,0,53.5,140.6,404.0
7,EFNB3-dimeric,NiV,0.117023,interpolated,0.117,0.366289,interpolated,0.366,1.272428,interpolated,1.27,0.117023,1.925617,1,0,117.0,366.3,1272.4


In [8]:
def plot_neut_curve(df):
    if receptor_flag:
        scale = alt.Scale(type='log')
        axis = alt.Axis(format='.0e',tickCount=3)
        title = 'Concentration (µM)'
        legend_title = 'Receptor'
    if sera_flag:
        scale = alt.Scale(type='log')
        axis = alt.Axis(format='.0e',tickCount=3)
        title = 'Sera Dilution'
        legend_title = 'Serum'
    if antibody_flag:
        scale = alt.Scale(type='log')
        axis = alt.Axis(format='.0e',tickCount=3)
        title = 'Concentration (µg/mL)'
        legend_title = 'Antibody'
    if vary_serum_flag:
        color_variable = 'serum'
    
    chart = (
        alt.Chart(df)
        .mark_line(size=1.5)
        .encode(
            x=alt.X(
                "concentration:Q",
                scale=scale,
                axis=axis,
                title=title,
            ),
            y=alt.Y(
                "fit:Q",
                title="Fraction Infectivity",
            ),
            color=alt.Color(color_variable, title=legend_title),
        )
    )
    circle = (
        alt.Chart(df)
        .mark_circle(size=40,opacity=1)
        .encode(
            x=alt.X(
                "concentration",
                scale=scale,
                axis=axis,
                title=title,
            ),
            y=alt.Y("measurement:Q", title="Fraction Infectivity"),
            color=alt.Color(color_variable, title=legend_title),
        )
    )
    error = (
        alt.Chart(df)
        .mark_errorbar(opacity=1)
        .encode(
            x="concentration",
            y=alt.Y("lower", title="Fraction Infectivity"),
            y2="upper",
            color=color_variable,
        )
    )
    plot = chart + circle + error
    plot = plot.properties(width=300,height=200)
    return plot


ephrin_curve = plot_neut_curve(neutcurve_df)
ephrin_curve.display()
ephrin_curve.save(output_png_path,ppi=300)

alt.LayerChart(...)